In [1]:
import numpy as np

In [13]:
def loadDataSet(fileName):
    dataMat =[]
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split("\t")
        fltLine = list(map(float,curLine))
        dataMat.append(fltLine)
    return dataMat

In [14]:
fileName ="testSet.txt"
dataList = loadDataSet(fileName)

In [16]:
def distEclud(vecA,vecB):
    return np.sqrt(np.sum(np.power(vecA-vecB,2)))

In [18]:
def randCent(dataSet,k):
    n = dataSet.shape[1]
    centroids = np.mat(np.zeros((k,n)))
    for j in range(n):
        minJ = min(dataSet[:,j])
        rangJ = float(max(dataSet[:,j])-minJ)
        centroids[:,j] = minJ + rangJ * np.random.rand(k,1)
    return centroids
        

In [19]:
dataMat = np.mat(dataList)
min(dataMat[:,1])

matrix([[-4.232586]])

In [22]:
print(dataMat[0])
print(dataMat[0,:])

[[ 1.658985  4.285136]]
[[ 1.658985  4.285136]]


In [20]:
randCent(dataMat,2)

matrix([[ 0.89464189,  3.59867419],
        [-3.98522945,  2.18740526]])

In [21]:
distEclud(dataMat[0],dataMat[1])

5.184632816681332

In [26]:
def KMeans(dataSet,k,distMeans = distEclud,createCent = randCent):
    m = dataSet.shape[0]
    clusterAssment = np.mat(np.zeros((m,2)))
    centroids = createCent(dataSet,k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = np.inf ; minIndex = -1
            for j in range(k):
                distJI = distMeans(dataSet[i,:],centroids[j,:])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            if clusterAssment[i,0] != minIndex : 
                clusterChanged = True
            clusterAssment[i,:] = minIndex, minDist **2
        print(centroids)
        for cent in range(k):
            ptsInClust = dataSet[np.nonzero(clusterAssment[:,0].A == cent)[0]]
            centroids[cent,:] = np.mean(ptsInClust,axis=0)
    return centroids ,clusterAssment

In [28]:
myCentroids ,clustAssing = KMeans(dataMat,4)

[[ 2.76601941  0.08400811]
 [-2.87109464  3.84112116]
 [ 0.86722025 -0.19362667]
 [-3.65402242  2.52736916]]
[[ 3.01944982  0.40588112]
 [-1.85352157  3.26866   ]
 [-1.61953947 -2.74384563]
 [-3.65788207 -0.19129671]]
[[ 3.00109112  0.05348327]
 [-1.66751027  3.1686925 ]
 [-2.35258211 -3.16690544]
 [-4.04190857 -1.446827  ]]
[[ 2.99405094 -0.1605263 ]
 [-1.6334182   3.03655888]
 [-2.37454129 -3.29526836]
 [-4.12671313 -2.51734387]]
[[ 2.99405094 -0.1605263 ]
 [-1.6334182   3.03655888]
 [-1.7671259  -3.2956472 ]
 [-4.04883533 -2.77633633]]
[[ 2.99405094 -0.1605263 ]
 [-1.6334182   3.03655888]
 [-1.46335425 -3.40428925]
 [-3.89646064 -2.78844243]]
[[ 2.99405094 -0.1605263 ]
 [-1.6334182   3.03655888]
 [-1.05904467 -3.687141  ]
 [-3.74393844 -2.75935387]]
[[ 3.02935331 -0.06628309]
 [-1.6334182   3.03655888]
 [ 0.1481304  -3.7396762 ]
 [-3.61853111 -2.81946867]]
[[ 3.09104296  1.05927767]
 [-1.6334182   3.03655888]
 [ 2.1347005  -3.50079317]
 [-3.53973889 -2.89384326]]
[[ 2.8962499   2.36

In [37]:
def biKMeans(dataMat,k,distMeas = distEclud):
    m = dataMat.shape[0]
    clusterAssment = np.mat(np.zeros((m,2)))
    centroid0 = np.mean(dataMat,axis=0).tolist()[0]
    centList = [centroid0]
    for j in range(m):
        clusterAssment[j,1] = distMeas(np.mat(centroid0),dataMat[j,:]) **2
    while(len(centList)<k):
        lowestSSE = np.inf
        for i in range(len(centList)):
            ptsInCurrentCluster = dataMat[np.nonzero(clusterAssment[:,0].A==i)[0],:]
            centroidMat,splitClustAss = KMeans(ptsInCurrentCluster,2,distMeas)
            sseSplit = np.sum(splitClustAss[:,1])
            sseNotSplit = np.sum(clusterAssment[np.nonzero(clusterAssment[:,0].A !=i)[0],1])
            print("sseSplit and sseNotSplit :" ,sseSplit,sseNotSplit)
            if(sseSplit + sseNotSplit) < lowestSSE:
                lowestSSE = sseSplit + sseNotSplit
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
        bestClustAss[np.nonzero(bestClustAss[:,0].A== 1)[0],0] = len(centList)
        bestClustAss[np.nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        centList[bestCentToSplit] = bestNewCents[0,:]
        centList.append(bestNewCents[1,:])
        clusterAssment[np.nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:] = bestClustAss
    return centList,clusterAssment

In [38]:
centList , myNewClustAssment = biKMeans(dataMat,3)

[[-3.76994231  0.82670627]
 [-3.61128678  3.52743566]]
[[-0.24627267 -2.28661917]
 [ 0.08939665  3.22142876]]
[[-0.2897198  -2.83942545]
 [ 0.08249337  2.94802785]]
sseSplit and sseNotSplit : 792.916856537 0.0
[[ 2.33708241 -0.94865242]
 [-1.63406295 -2.78574888]]
[[ 3.03713839 -2.62802833]
 [-3.01169468 -3.01238673]]
[[ 2.80293085 -2.7315146 ]
 [-3.38237045 -2.9473363 ]]
sseSplit and sseNotSplit : 83.5874695564 326.284075201
[[-3.53496873  1.44396209]
 [ 2.51761207  1.30012101]]
[[-2.46154315  2.78737555]
 [ 2.6265299   3.10868015]]
sseSplit and sseNotSplit : 66.36683512 466.632781336
